<a href="https://colab.research.google.com/github/Dancingtree95/AlfaBattle2.0/blob/master/AlfaTransactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%%capture
!wget https://storage.yandexcloud.net/ds-ods/files/materials/6e991b7f/train_transactions_contest.zip
!wget https://storage.yandexcloud.net/ds-ods/files/materials/fc0f1aa3/test_transactions_contest.zip
!wget https://storage.yandexcloud.net/ds-ods/files/materials/3634ff95/train_target.csv
!wget https://storage.yandexcloud.net/ds-ods/files/materials/b2216108/test_target_contest.csv

In [5]:
%%capture
!unzip train_transactions_contest.zip
!rm train_transactions_contest.zip

In [1]:
!git clone https://github.com/smirnovevgeny/AlfaBattle2.0.git

Cloning into 'AlfaBattle2.0'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 57 (delta 22), reused 35 (delta 8), pack-reused 0
Unpacking objects: 100% (57/57), done.


In [7]:
import os
import pandas as pd
import sys
import pickle
import numpy as np
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [8]:
sys.path.append('AlfaBattle2.0')
sys.path.append('AlfaBattle2.0/rnn_baseline')
sys.path.append('AlfaBattle2.0/rnn_baseline/baseline')

In [9]:
TRAIN_TRANSACTIONS_PATH = 'train_transactions_contest'
TRAIN_TARGET_PATH = 'train_target.csv'

In [10]:
target_frame = pd.read_csv(TRAIN_TARGET_PATH)
target_frame.head()

,app_id,product,flag
0,0,3,0
1,1,1,0
2,2,1,0
3,3,1,0
4,4,1,0


In [11]:
from utils import read_parquet_dataset_from_local
from dataset_preprocessing_utils import transform_transactions_to_sequences, create_padded_buckets

In [2]:
import pickle

with open('/content/AlfaBattle2.0/rnn_baseline/constants/buckets_info.pkl', 'rb') as f:
    mapping_seq_len_to_padded_len = pickle.load(f)
    
with open('/content/AlfaBattle2.0/rnn_baseline/constants/dense_features_buckets.pkl', 'rb') as f:
    dense_features_buckets = pickle.load(f)

with open('/content/AlfaBattle2.0/rnn_baseline/constants/embedding_projections.pkl', 'rb') as f:
  embedding_projections = pickle.load(f)

In [ ]:
def create_buckets_from_transactions(path_to_dataset, save_to_path, frame_with_ids = None, 
                                     num_parts_to_preprocess_at_once: int = 1, 
                                     num_parts_total=50, has_target=False):
    block = 0
    for step in tqdm(range(0, num_parts_total, num_parts_to_preprocess_at_once), 
                                   desc="Transforming transactions data"):
        transactions_frame = read_parquet_dataset_from_local(path_to_dataset, step, num_parts_to_preprocess_at_once, 
                                                             verbose=True)
        for dense_col in ['amnt', 'days_before', 'hour_diff']:
            transactions_frame[dense_col] = np.digitize(transactions_frame[dense_col], bins=dense_features_buckets[dense_col])
            
        seq = transform_transactions_to_sequences(transactions_frame)
        seq['sequence_length'] = seq.sequences.apply(lambda x: len(x[1]))
        
        if frame_with_ids is not None:
            seq = seq.merge(frame_with_ids, on='app_id')

        block_as_str = str(block)
        if len(block_as_str) == 1:
            block_as_str = '00' + block_as_str
        else:
            block_as_str = '0' + block_as_str
            
        processed_fragment =  create_padded_buckets(seq, mapping_seq_len_to_padded_len, has_target=has_target, 
                                                    save_to_file_path=os.path.join(save_to_path, 
                                                                                   f'processed_chunk_{block_as_str}.pkl'))
        block += 1